In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
import json

##cookies

class Test_selenimu1:
    def setup(self):
        service = Service(r"D://chromedriver//chromedriver-win64//chromedriver.exe")##换成自己的chromedriver路径
        self.driver = webdriver.Chrome(service=service)
        self.driver.get(r"https://douban.com")

    @pytest.mark.skip(msg="先不用")
    def test_cookies(self):
        driver = self.driver
        driver.implicitly_wait(60)
        iframe = driver.find_element(By.TAG_NAME, "iframe")
        driver.switch_to.frame(iframe)
        driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/ul[1]/li[2]").click()
        driver.find_element(By.ID, "username").send_keys("***********")
        driver.find_element(By.ID, "password").send_keys("***********")
        #等待60秒，手动登陆
        sleep(60)

        cookies = driver.get_cookies()

        with open(r"E://py-practice//cookie.txt", 'w') as file1:
            for cookie in cookies:
                file1.write(json.dumps(cookie) + "\n")

    # @pytest.mark.skip(msg="先不用")
    def test_case2(self):
        driver = self.driver
        driver.get(r"https://douban.com")
        driver.implicitly_wait(10)
        with open(r"E://py-practice//cookie.txt", 'r') as file2:
            res = file2.readlines()
            for line in res:
                # print(line)
                cookie = json.loads(line)
                # print(cookie)
                if "expiry" in cookie:
                    del cookie["expiry"]
                driver.add_cookie(cookie)

            # for line in file2.readlines():
            #     res = line.strip()
            #     print(res)
            #     mycookie = json.loads(res)
            #     print(mycookie)
            #     driver.add_cookie(mycookie)
             # res = file2.readline()
             # mycookies = json.loads(res)
        driver.refresh()
        sleep(3)
    def teardown(self):
        self.driver.quit()

##运行
# 创建类实例并调用方法
test_instance = Test_selenimu1()
test_instance.setup()
test_instance.test_cookies()  # 或 test_instance.test_case2()
test_instance.teardown()

In [ ]:
import requests
from time import sleep
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

# 禁用安全请求警告
requests.packages.urllib3.disable_warnings()

header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
    'cookie': '$$填自己的cookie',
}

def get_page(i, base_url):
    listcnt = 25
    start = listcnt * i
    params = {
        'start': start,
        'type': 'new'
    }
    try:
        response = requests.get(base_url, headers=header, params=params, verify=False)
        if response.status_code == 200:
            return response.text
    except requests.ConnectionError:
        print('Error in ', base_url + str(params))
        return None

def get_discussion_list(base_url):
    df = pd.DataFrame(columns=['title', 'elite', 'url', 'author-name', 'author-url', 'r-count', 'time', 'page', 'rank', 'timestamp'])

    for i in range(0, 241):##爬取页数
        print('page:', i + 1)
        res = get_page(i, base_url)
        soup = BeautifulSoup(res, 'lxml')
        attrs = {'class': 'olt'}
        raw_tb = soup.find(attrs=attrs).find_all('tr')
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        for rk, tr in enumerate(raw_tb[1:]):
            col = {}
            tds = tr.find_all('td')
            col['title'] = tds[0].find('a').get('title')
            col['elite'] = 1 if tds[0].find(attrs={'class': 'elite_topic_lable'}) else 0
            col['url'] = tds[0].find('a').get('href')
            col['author-name'] = tds[1].find('a').get_text()
            col['author-url'] = tds[1].find('a').get('href')
            col['r-count'] = tds[2].get_text()
            col['time'] = tds[3].get_text()
            col['page'] = i + 1
            col['rank'] = rk + 1
            col['timestamp'] = timestamp
            df = pd.concat([df, pd.DataFrame([col])], ignore_index=True)
        sleep(3)
    return df

if __name__ == '__main__':
    grouplist_url = '$$填要爬的小组url'
    list_df = get_discussion_list(grouplist_url)
    list_df.drop_duplicates(subset=['title', 'url'], inplace=True)
    list_df.to_csv('E://py-practice//discussion_list.csv', index=False, encoding='utf-8-sig')##保存成文件